# Lab 1: Mixtures of Experts

This notebook demonstrates how mixtures of experts can be used to boost performance.

The objective of this lab is to classify images from Cifar10 (https://www.cs.toronto.edu/~kriz/cifar.html) to one of ten classes: {0: airplane, 1: automobile, 2: bird, 3: cat, 4: deer, 5: dog, 6: frog, 7: horse, 8: ship, 9: truck}


![Cifar10](cifar10_resize.png )

Specifically, a gating function is trained to pass examples to two experts which are trained separatly, where one expert is trained to classify images within the "natural image" category (e.g. cat, dog, etc) and another to classify images within the  "artificial image" category (e.g. plane, car). The experts are then used to boost the performance of a baseline architecture that classifies image to one of the 10 classes.

Specifically, the mixture is built in the following order:
1. A single model is trained to to classify all 10 classes. (This is included in the mixture, and is also our evaluation benchmark)

2. An expert gating function is trained to recognise whether an image is of an artificial or  natural subject.

3. An artificial expert is trained to classify artificial objects that have a label in {0, 1, 8, 9}.

4. A natural expert is trained to classify natural objects that have a label in  {2, 3, 4, 5, 6, 7}.

5. A gating function is trained to determine the contribution of the experts and the contribution of the baseline architecture to the final output.

6. The mixture is built as illustrated in the figure below.


![](moe_architecture_illus.png)

## Import Prerequisites

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import concatenate, Lambda, Reshape
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import keras.backend as K
from keras.datasets import cifar10
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import numpy as np
import os
import copy
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import pydot
from IPython.display import SVG


In [ ]:
# Parameters (not to be changed)
orig_classes = 10
gate0_classes = 2


## Mixture Parameters

You can try changing the mixture parameters in the following piece of code, when doing so, consider the following:

1 - Increasing the number of epochs increases the fit to training data, at some point this should cause over-fitting. Conversly, setting it low should cause under-fitting.

2 - Increasing the number of training examples increases the number of learnable features.

3 - Using a large model for different classifiers increases their capacity to learn. This increases the amount of required epochs for training, and also increases the risk of over-fitting.

By changing the parameters, the performance of the mixture of experts and the baseline classifier should change accordingly.

In [ ]:
# Number of training/testing examples per batch
batch_size = 50

# Training epochs. A higher number of epochs corresponds to "more fitting to training data"
epochs = 1

# Number of training/testing examples to use
train_examples = 5000  # Max is 50000
test_examples = 1000  # Max is 5000

# Large/small model flags. Set to true to change a classifier to "large"
use_large_experts = False
use_large_gating_mlp = False
use_large_baseline_classifier = False


In [ ]:
# delete previous model checkpoints
import shutil

shutil.rmtree("gate0Cifar10", ignore_errors=True)
shutil.rmtree("moe3Cifar10", ignore_errors=True)
shutil.rmtree("natureCifar10", ignore_errors=True)
shutil.rmtree("baseCifar10", ignore_errors=True)
shutil.rmtree("artCifar10", ignore_errors=True)


# get the newest model file within a directory
def getNewestModel(model, dirname):
    from glob import glob

    target = os.path.join(dirname, "*")
    files = [(f, os.path.getmtime(f)) for f in glob(target)]
    if len(files) == 0:
        return model
    else:
        newestModel = sorted(files, key=lambda files: files[1])[-1]
        model.load_weights(newestModel[0])
        return model


## Prepare datasets

In [ ]:
# load dataset  ; X: input images,  Y: class label ground truth
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train[:train_examples]
x_test = x_test[:test_examples]
y_train = y_train[:train_examples]
y_test = y_test[:test_examples]


In [ ]:
# prepare x dataset
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255


In [ ]:
# Convert class vectors to binary class matrices
y_train0 = keras.utils.to_categorical(y_train, orig_classes)
y_test0 = keras.utils.to_categorical(y_test, orig_classes)

print("y train0:{0}\ny test0:{1}".format(y_train0.shape, y_test0.shape))


## Define architectures

In [ ]:
# input layer
cifarInput = Input(shape=(x_train.shape[1:]), name="input")


In [ ]:
# Small VGG-like model
def simpleVGG(cifarInput, num_classes, name="vgg"):
    name = [name + str(i) for i in range(12)]

    # convolution and max pooling layers
    vgg = Conv2D(32, (3, 3), padding="same", activation="relu", name=name[0])(
        cifarInput
    )
    vgg = Conv2D(32, (3, 3), padding="same", activation="relu", name=name[1])(vgg)
    vgg = MaxPooling2D(pool_size=(2, 2), name=name[2])(vgg)
    vgg = Dropout(0.25, name=name[3])(vgg)
    vgg = Conv2D(64, (3, 3), padding="same", activation="relu", name=name[4])(vgg)
    vgg = Conv2D(64, (3, 3), padding="same", activation="relu", name=name[5])(vgg)
    vgg = MaxPooling2D(pool_size=(2, 2), name=name[6])(vgg)
    vgg = Dropout(0.25, name=name[7])(vgg)

    # classification layers
    vgg = Flatten(name=name[8])(vgg)
    vgg = Dense(512, activation="relu", name=name[9])(vgg)
    vgg = Dropout(0.5, name=name[10])(vgg)
    vgg = Dense(num_classes, activation="softmax", name=name[11])(vgg)
    return vgg


In [ ]:
# Large VGG-like model
def fatVGG(cifarInput, num_classes, name="vgg"):
    name = [name + str(i) for i in range(17)]

    # convolution and max pooling layers
    vgg = Conv2D(32, (3, 3), padding="same", activation="relu", name=name[0])(
        cifarInput
    )
    vgg = Conv2D(32, (3, 3), padding="same", activation="relu", name=name[1])(vgg)
    vgg = MaxPooling2D(pool_size=(2, 2), name=name[2])(vgg)
    vgg = Dropout(0.25, name=name[3])(vgg)
    vgg = Conv2D(64, (3, 3), padding="same", activation="relu", name=name[4])(vgg)
    vgg = Conv2D(64, (3, 3), padding="same", activation="relu", name=name[5])(vgg)
    vgg = MaxPooling2D(pool_size=(2, 2), name=name[6])(vgg)
    vgg = Dropout(0.25, name=name[7])(vgg)
    vgg = Conv2D(128, (3, 3), padding="same", activation="relu", name=name[8])(vgg)
    vgg = Conv2D(128, (3, 3), padding="same", activation="relu", name=name[9])(vgg)
    vgg = Conv2D(128, (3, 3), padding="same", activation="relu", name=name[10])(vgg)
    vgg = MaxPooling2D(pool_size=(2, 2), name=name[11])(vgg)
    vgg = Dropout(0.25, name=name[12])(vgg)

    # classification layers
    vgg = Flatten(name=name[13])(vgg)
    vgg = Dense(512, activation="relu", name=name[14])(vgg)
    vgg = Dropout(0.5, name=name[15])(vgg)
    vgg = Dense(num_classes, activation="softmax", name=name[16])(vgg)
    return vgg


In [ ]:
# first gating network, to decide artificial or natural object
if use_large_gating_mlp:
    gate0VGG = fatVGG(cifarInput, gate0_classes, "gate0")
else:
    gate0VGG = simpleVGG(cifarInput, gate0_classes, "gate0")
gate0Model = Model(cifarInput, gate0VGG)

# base VGG
if use_large_baseline_classifier:
    baseVGG = fatVGG(cifarInput, orig_classes, "base")
else:
    baseVGG = simpleVGG(cifarInput, orig_classes, "base")
baseModel = Model(cifarInput, baseVGG)

# artificial expert VGG
if use_large_experts:
    artificialVGG = fatVGG(cifarInput, orig_classes, "artificial")
else:
    artificialVGG = simpleVGG(cifarInput, orig_classes, "artificial")
artificialModel = Model(cifarInput, artificialVGG)

# naturalVGG = fatVGG(cifarInput, orig_classes, "natural")
if use_large_experts:
    naturalVGG = fatVGG(cifarInput, orig_classes, "natural")
else:
    naturalVGG = simpleVGG(cifarInput, orig_classes, "natural")

naturalModel = Model(cifarInput, naturalVGG)


## Train networks

### Train 10-Class Classifier

In [ ]:
# compile
baseModel.compile(
    loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"]
)


In [ ]:
# make saving directory for checkpoints
baseSaveDir = "./baseCifar10/"
if not os.path.isdir(baseSaveDir):
    os.makedirs(baseSaveDir)

# early stopping and model checkpoint
es_cb = EarlyStopping(monitor="val_loss", patience=2, verbose=1, mode="auto")
chkpt = os.path.join(baseSaveDir, "Cifar10_.{epoch:02d}-{val_loss:.2f}.hdf5")
cp_cb = ModelCheckpoint(
    filepath=chkpt, monitor="val_loss", verbose=1, save_best_only=True, mode="auto"
)

# load the newest model data from the directory if exists
baseModel = getNewestModel(baseModel, baseSaveDir)


In [ ]:
# train
baseModel.fit(
    x_train,
    y_train0,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test0),
    callbacks=[es_cb, cp_cb],
)


In [ ]:
# evaluate
baseModel = getNewestModel(baseModel, baseSaveDir)
baseScore = baseModel.evaluate(x_test, y_test0)
print(baseScore)


## Train 2-Class Natural/Artificial Classifier

The expert gating model determines whether an image is "natural" or "artificial".

In [ ]:
# Make ground truth for whether an example is "natural" or "artificial"
y_trainG0 = np.array([0 if i in [0, 1, 8, 9] else 1 for i in y_train])
y_testG0 = np.array([0 if i in [0, 1, 8, 9] else 1 for i in y_test])

y_trainG0 = keras.utils.to_categorical(y_trainG0, 2)
y_testG0 = keras.utils.to_categorical(y_testG0, 2)

print("y trainG0:{0}\ny testG0:{1}".format(y_trainG0.shape, y_testG0.shape))


In [ ]:
# compile
gate0Model.compile(
    loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"]
)


In [ ]:
# make saving directory for check point
gate0SaveDir = "./gate0Cifar10/"
if not os.path.isdir(gate0SaveDir):
    os.makedirs(gate0SaveDir)

# early stopping and model checkpoint
es_cb = EarlyStopping(monitor="val_loss", patience=2, verbose=1, mode="auto")
chkpt = os.path.join(gate0SaveDir, "Cifar10_.{epoch:02d}-{val_loss:.2f}.hdf5")
cp_cb = ModelCheckpoint(
    filepath=chkpt, monitor="val_loss", verbose=1, save_best_only=True, mode="auto"
)

# load the newest model data from the directory if exists
gate0Model = getNewestModel(gate0Model, gate0SaveDir)


In [ ]:
# train
gate0Model.fit(
    x_train,
    y_trainG0,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_testG0),
    callbacks=[es_cb, cp_cb],
)


In [ ]:
# evaluate
gate0Model = getNewestModel(gate0Model, gate0SaveDir)
gate0Score = gate0Model.evaluate(x_test, y_testG0)
print(gate0Score)


## Train "Natural" and "Artificial" Experts
<br>
The expert networks are specialized in predicting a certain classes.<br>
Each network is only trained with its specialized field: the artificial expert get trained for labels 0, 1, 8 and 9; the natural expert for labels 2, 3, 4, 5, 6 and 7.

In [ ]:
# get the position of artificial images and natural images in training and test dataset
artTrain = [i for i in range(len(y_train)) if y_train[i] in [0, 1, 8, 9]]
natureTrain = [i for i in range(len(y_train)) if y_train[i] in [2, 3, 4, 5, 6, 7]]
artTest = [i for i in range(len(y_test)) if y_test[i] in [0, 1, 8, 9]]
natureTest = [i for i in range(len(y_test)) if y_test[i] in [2, 3, 4, 5, 6, 7]]


In [ ]:
# get artificial dataset and natural dataset
x_trainArt = x_train[artTrain]
x_testArt = x_test[artTest]
y_trainArt = y_train[artTrain]
y_testArt = y_test[artTest]


### Artificial expert network

In [ ]:
# for artificial dataset
y_trainArt = keras.utils.to_categorical(y_trainArt, orig_classes)
y_testArt = keras.utils.to_categorical(y_testArt, orig_classes)

print("y train art:{0}\ny test art:{1}".format(y_trainArt.shape, y_testArt.shape))


In [ ]:
# compile
artificialModel.compile(
    loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"]
)


In [ ]:
# make saving directory for check point
artSaveDir = "./artCifar10/"
if not os.path.isdir(artSaveDir):
    os.makedirs(artSaveDir)

# early stopping and model checkpoint
es_cb = EarlyStopping(monitor="val_loss", patience=2, verbose=1, mode="auto")
chkpt = os.path.join(artSaveDir, "Cifar10_.{epoch:02d}-{val_loss:.2f}.hdf5")
cp_cb = ModelCheckpoint(
    filepath=chkpt, monitor="val_loss", verbose=1, save_best_only=True, mode="auto"
)

# load the newest model data if exists
artificialModel = getNewestModel(artificialModel, artSaveDir)


In [ ]:
# train
artificialModel.fit(
    x_trainArt,
    y_trainArt,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_testArt, y_testArt),
    callbacks=[es_cb, cp_cb],
)


In [ ]:
# evaluate
artificialModel = getNewestModel(artificialModel, artSaveDir)
artScore = artificialModel.evaluate(x_testArt, y_testArt)
print(artScore)


### Natural expert network

In [ ]:
# for natural dataset
x_trainNat = x_train[natureTrain]
x_testNat = x_test[natureTest]
y_trainNat = y_train[natureTrain]
y_testNat = y_test[natureTest]


In [ ]:
# get natural dataset
y_trainNat = keras.utils.to_categorical(y_trainNat, orig_classes)
y_testNat = keras.utils.to_categorical(y_testNat, orig_classes)

print("y train nature:{0}\ny test nature:{1}".format(y_trainNat.shape, y_testNat.shape))


In [ ]:
# compile
naturalModel.compile(
    loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"]
)


In [ ]:
# make saving directory for check point
natSaveDir = "./natureCifar10/"
if not os.path.isdir(natSaveDir):
    os.makedirs(natSaveDir)

# early stopping and model checkpoint
es_cb = EarlyStopping(monitor="val_loss", patience=2, verbose=1, mode="auto")
chkpt = os.path.join(natSaveDir, "Cifar10_.{epoch:02d}-{val_loss:.2f}.hdf5")
cp_cb = ModelCheckpoint(
    filepath=chkpt, monitor="val_loss", verbose=1, save_best_only=True, mode="auto"
)

# load the newest model data if exists
naturalModel = getNewestModel(naturalModel, natSaveDir)


In [ ]:
# train
naturalModel.fit(
    x_trainNat,
    y_trainNat,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_testNat, y_testNat),
    callbacks=[es_cb, cp_cb],
)


In [ ]:
# evaluate
naturalModel = getNewestModel(naturalModel, natSaveDir)
natScore = naturalModel.evaluate(x_testNat, y_testNat)
print(natScore)


### Freeze the weights of all trained models so far (i.e. baseline, experts, and expert gating models).


In [ ]:
for l in baseModel.layers:
    l.trainable = False
for l in gate0Model.layers:
    l.trainable = False
for l in artificialModel.layers:
    l.trainable = False
for l in naturalModel.layers:
    l.trainable = False


## Connecting the overall networks to form the mixture of experts model

Sub-gate

Up to this point, we have a baseline classifier which classifies the 10 classes, we have 2 experts which each classify the 10 classes but specialise in classifying either the natural or artificial categories of classes, and we have a first gate which decides which of the experts to choose for the given cifar input. We want our second gate to be able to take in the cifar input and decide what the importance should be of the output of the i) baseline and ii) chosen expert, thereby determining the importance of the baseline and the chosen expert in producing a final MoE output prediction. To do this, the second gate will be composed of 2 sub-gates; one for each expert.

*Sub-gate Structure*: First layer of sub-gate is our (32, 32, 3) input layer as before which takes in the cifar data -> flatten input -> pass through 512 unit dense layer with relu activation function -> pass through dropout layer which randomly sets units to 0 with probability 0.5 -> pass through orig_classes x 2 = 10 x 2 = 20 unit dense layer with softmax activation function -> now have importance values for the i) baseline and ii) chosen expert -> reshape this 1D output into (10, 2) output. I.e. along [:, 0], have a (10,) tensor of what the sub-gate 'thinks' the importance is of the 10 baseline classifier outputs, and along [:,1], have a (10,) tensor of what the sub-gate 'thinks' the importance is of the expert.

Instantiate one of these sub-gates for each expert. Whichever expert is chosen by the first binary classifier 'hard' gate will determine which corresponding expert sub-gate is used to 'soft gate' between the baseline and the chosen expert.

In [ ]:
# define sub-Gate network, for the second gating network layer
def subGate(cifarInput, orig_classes, name="subGate"):
    name = [name + str(i) for i in range(5)]
    subgate = Flatten(name=name[0])(cifarInput)
    subgate = Dense(512, activation="relu", name=name[1])(subgate)
    subgate = Dropout(0.5, name=name[2])(subgate)
    subgate = Dense(orig_classes * 2, activation="softmax", name=name[3])(subgate)
    subgate = Reshape((orig_classes, 2), name=name[4])(subgate)
    return subgate


In [ ]:
# the artificial gating network
artGate = subGate(cifarInput, orig_classes, "artExpertGate")

# the natural gating network
natureGate = subGate(cifarInput, orig_classes, "natureExpertGate")


Sub-gate Lambda

Want this layer to take outputs of i) baseline ii) expert and iii) sub-gate of the corresponding expert -> get logit outputs for each of the 10 orig_classes

- Takes in argument gx, which is a list of 3 tensors (the output of i) baseline ii) expert iii) corresponding expert sub-gate)

- gx[0] -> baseline output tensor (10,). Is a softmax output for each of the 10 classes

- gx[1] -> expert network output tensor (10,). Is a softmax output for each of the 10 classes. Which expert's output reached here depends on the binary classifier output of the first gate (we will implement the logic of choosing the expert when we tie all the models together, see below).

- gx[2] -> corresponding expert sub-gate output tensor (10,2,). 
- gx[2][:,:,0] -> baseline importance tensor of shape (10,). Is what the sub-gate thinks the importance is of each of the 10 baseline output classes. 
- gx[2][:,:,1] -> expert importance tensor of shape (10,). Is what the sub-gate thinks the importance is of each of the 10 expert output classes.

We ultimately want a logit output for each of the 10 classes. We want this output to be determined by what the i) baseline and ii) expert thought, weighted by the importance of what the sub-gate thought. To do this inference, we can define a simple function which: i) multiplies the baseline's output by the sub-gate's baseline importance -> get a (10,) tensor ii) multiplies the expert's output by the sub-gate's expert importance -> get a (10,) tensor iii) sum these two importance-weighted terms to get a final (10,) tensor of logit outputs (one for each class).

In [ ]:
# define inference calculation with Keras Lambda layer with base VGG, expert network and the second gating network of corresponding expert as input
# the inference is calculated as sum of multiplications of base VGG inference output and its importance, and expert network inference output and its importance
def subGateLambda(base, expert, subgate):
    output = Lambda(
        lambda gx: (gx[0] * gx[2][:, :, 0]) + (gx[1] * gx[2][:, :, 1]),
        output_shape=(orig_classes,),
    )([base, expert, subgate])
    return output


# DEBUG
# def subGateLambda(base, expert, subgate):
#     output = Lambda(lambda gx: print('\ngx: {}\ngx[0]: {}\ngx[2][:,:,0]: {}\ngx[1]: {}\ngx[2][:,:,1]: {}'.format(gx, gx[0],gx[2][:,:,0],gx[1],gx[2][:,:,1])))([base, expert, subgate])
#     return output


Connecting the Networks

Now we just need to i) tie all of the above models together and ii) implement the logic for choosing an expert at the first gate. The first 'expert gate' binary classifier is a 'hard gate' since it will choose one expert and block the other. The second gate (a sub-gate) is a 'soft gate' since it will apply its importance weights across the i) baseline and ii) chosen expert.

To do this, we can define a layer which takes the outputs of i) baseline ii) first gate iii) artificial expert iv) natural expert v) artificial sub-gate vi) natural sub-gate.

- Takes in argument gx, which is a list of 6 tensors (the output of i) baseline ii) first gate iii) artificial expert iv) natural expert v) artificial sub-gate vi) natural sub-gate)
- gx[0] -> baseline output tensor (10,)
- gx[1] -> first gate binary output tensor (2,)
- gx[2] -> artificial expert output tensor (10,)
- gx[3] -> natural expert output tensor (10,)
- gx[4] -> artificial sub-gate output tensor (10,2,)
- gx[5] -> natural sub-gate output tensor (10,2,)

We want to implement the logic that the first gate's chosen expert's output and corresponding sub-gate output should be passed (along with the baseline's output) to the sub-gate lambda function defined previously. To do this, we can use the Keras backend switch function, which will pass the chosen expert's output and its corresponding sub-gate output to the sub-gate lambda depending on which of the 2 outputs of the first gate was greater.

In [ ]:
import tensorflow as tf

# connecting the overall networks.
# the Keras backend switch works as deciding with the first gating network, leading to artificial or natural gate
output = Lambda(
    lambda gx: K.switch(
        tf.expand_dims(gx[1][:, 0], axis=1) > tf.expand_dims(gx[1][:, 1], axis=1),
        subGateLambda(gx[0], gx[2], gx[4]),
        subGateLambda(gx[0], gx[3], gx[5]),
    ),
    output_shape=(orig_classes,),
)([baseVGG, gate0VGG, artificialVGG, naturalVGG, artGate, natureGate])

# DEBUG
# output = Lambda(lambda gx: print('\ngx: {}\ngx[0]: {}\ngx[1]: {}\ngx[2]: {}\ngx[3]: {}\ngx[4]: {}\ngx[5]: {}'.format(gx, gx[0], gx[1], gx[2], gx[3], gx[4], gx[5])),
#                 output_shape=(orig_classes,))([baseVGG, gate0VGG, artificialVGG, naturalVGG, artGate, natureGate])


In [ ]:
# the mixture of experts model
model = Model(cifarInput, output)


In [ ]:
# compile
model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])


At this point we have already trained the baseline, the experts, and the first expert gate. Now we need only train the sub-gates and the final Lambda inference layer, which will take the outputs of the above trained models and learn to output a final (10,) MoE prediction. 

In [ ]:
# show layers and if it's trainable or not
# only the second gating network layers and the last Lambda inference layer are left trainable
for l in model.layers:
    print(l, l.trainable)


In [ ]:
# make saving directory for check point
saveDir = "./moe3Cifar10/"
if not os.path.isdir(saveDir):
    os.makedirs(saveDir)

# early stopping and model checkpoint
es_cb = EarlyStopping(monitor="val_loss", patience=2, verbose=1, mode="auto")
chkpt = os.path.join(saveDir, "Cifar10_.{epoch:02d}-{val_loss:.2f}.hdf5")
cp_cb = ModelCheckpoint(
    filepath=chkpt, monitor="val_loss", verbose=1, save_best_only=True, mode="auto"
)

# load the newest model data if exists
model = getNewestModel(model, saveDir)


In [ ]:
# train
model.fit(
    x_train,
    y_train0,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test0),
    callbacks=[es_cb, cp_cb],
)


In [ ]:
# evaluate
mixture_loss_accuracy = model.evaluate(x_test, y_test0)
print(mixture_loss_accuracy)
